<a href="https://colab.research.google.com/github/ParishaKB/Hackathons/blob/main/job-a-thon/Model_Selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing 

In [3]:
train = pd.read_csv('/content/drive/MyDrive/Job-a-thon/train_Df64byy.csv')
test = pd.read_csv('/content/drive/MyDrive/Job-a-thon/test_YCcRUnU.csv')

In [4]:
print(train.shape,test.shape)

(50882, 14) (21805, 13)


## EDA

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       50882 non-null  int64  
 1   City_Code                50882 non-null  object 
 2   Region_Code              50882 non-null  int64  
 3   Accomodation_Type        50882 non-null  object 
 4   Reco_Insurance_Type      50882 non-null  object 
 5   Upper_Age                50882 non-null  int64  
 6   Lower_Age                50882 non-null  int64  
 7   Is_Spouse                50882 non-null  object 
 8   Health Indicator         39191 non-null  object 
 9   Holding_Policy_Duration  30631 non-null  object 
 10  Holding_Policy_Type      30631 non-null  float64
 11  Reco_Policy_Cat          50882 non-null  int64  
 12  Reco_Policy_Premium      50882 non-null  float64
 13  Response                 50882 non-null  int64  
dtypes: float64(2), int64(6

In [6]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21805 entries, 0 to 21804
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ID                       21805 non-null  int64  
 1   City_Code                21805 non-null  object 
 2   Region_Code              21805 non-null  int64  
 3   Accomodation_Type        21805 non-null  object 
 4   Reco_Insurance_Type      21805 non-null  object 
 5   Upper_Age                21805 non-null  int64  
 6   Lower_Age                21805 non-null  int64  
 7   Is_Spouse                21805 non-null  object 
 8   Health Indicator         16778 non-null  object 
 9   Holding_Policy_Duration  13202 non-null  object 
 10  Holding_Policy_Type      13202 non-null  float64
 11  Reco_Policy_Cat          21805 non-null  int64  
 12  Reco_Policy_Premium      21805 non-null  float64
dtypes: float64(2), int64(5), object(6)
memory usage: 2.2+ MB


In [7]:
train['Holding_Policy_Duration'].fillna(0, inplace=True)
train['Holding_Policy_Type'].fillna(0, inplace=True)
test['Holding_Policy_Duration'].fillna(0, inplace=True)
test['Holding_Policy_Type'].fillna(0, inplace=True)

In [8]:
train['Holding_Policy_Duration'] = train['Holding_Policy_Duration'].astype(str)
test['Holding_Policy_Duration'] = test['Holding_Policy_Duration'].astype(str)

In [9]:
bins = [2000,   6000, 10000,  14000,  18000,   22000, 26000,  30000,   34000, 38000, 42000, 46000 ]
labels = ['0', '1',   '2',    '3',    '4',    '5',    '6',    '7',    '8',    '9', '10']
train['prembin'] = pd.cut(train['Reco_Policy_Premium'], bins, labels = labels,include_lowest = True)
test['prembin'] = pd.cut(test['Reco_Policy_Premium'], bins, labels = labels,include_lowest = True)

In [10]:
train.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response,prembin
0,1,C3,3213,Rented,Individual,36,36,No,X1,14+,3.0,22,11628.0,0,2
1,2,C5,1117,Owned,Joint,75,22,No,X2,0,0.0,22,30510.0,0,7
2,3,C5,3732,Owned,Individual,32,32,No,NaN,1.0,1.0,19,7450.0,1,1
3,4,C24,4378,Owned,Joint,52,48,No,X1,14+,3.0,19,17780.0,0,3
4,5,C8,2190,Rented,Individual,44,44,No,X2,3.0,1.0,16,10404.0,0,2


In [11]:
test.head()

,ID,City_Code,Region_Code,Accomodation_Type,Reco_Insurance_Type,Upper_Age,Lower_Age,Is_Spouse,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,prembin
0,50883,C1,156,Owned,Individual,30,30,No,NaN,6.0,3.0,5,11934.0,2
1,50884,C4,7,Owned,Joint,69,68,Yes,X1,3.0,3.0,18,32204.8,7
2,50885,C1,564,Rented,Individual,28,28,No,X3,2.0,4.0,17,9240.0,1
3,50886,C3,1177,Rented,Individual,23,23,No,X3,3.0,3.0,18,9086.0,1
4,50887,C1,951,Owned,Individual,75,75,No,X3,0,0.0,5,22534.0,5


In [12]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   ID                       50882 non-null  int64   
 1   City_Code                50882 non-null  object  
 2   Region_Code              50882 non-null  int64   
 3   Accomodation_Type        50882 non-null  object  
 4   Reco_Insurance_Type      50882 non-null  object  
 5   Upper_Age                50882 non-null  int64   
 6   Lower_Age                50882 non-null  int64   
 7   Is_Spouse                50882 non-null  object  
 8   Health Indicator         39191 non-null  object  
 9   Holding_Policy_Duration  50882 non-null  object  
 10  Holding_Policy_Type      50882 non-null  float64 
 11  Reco_Policy_Cat          50882 non-null  int64   
 12  Reco_Policy_Premium      50882 non-null  float64 
 13  Response                 50882 non-null  int64   
 14  prembi

In [13]:
!pip install category_encoders

     |████████████████████████████████| 81kB 5.4MB/s 


In [14]:
x = train.drop(['ID','Response'],axis = 1)
y = train['Response']
test = test.drop(['ID'],axis = 1)

In [15]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   City_Code                50882 non-null  object  
 1   Region_Code              50882 non-null  int64   
 2   Accomodation_Type        50882 non-null  object  
 3   Reco_Insurance_Type      50882 non-null  object  
 4   Upper_Age                50882 non-null  int64   
 5   Lower_Age                50882 non-null  int64   
 6   Is_Spouse                50882 non-null  object  
 7   Health Indicator         39191 non-null  object  
 8   Holding_Policy_Duration  50882 non-null  object  
 9   Holding_Policy_Type      50882 non-null  float64 
 10  Reco_Policy_Cat          50882 non-null  int64   
 11  Reco_Policy_Premium      50882 non-null  float64 
 12  prembin                  50882 non-null  category
dtypes: category(1), float64(2), int64(4), object(6)
memory usage:

In [16]:
import category_encoders as ce
encoder=ce.TargetEncoder() 
x_train_encoded = encoder.fit_transform(x, y)
test_encoded = encoder.transform(test)

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [17]:
print(x_train_encoded.shape,y.shape,test_encoded.shape)

(50882, 13) (50882,) (21805, 13)


In [18]:
x_train_encoded.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50882 entries, 0 to 50881
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   City_Code                50882 non-null  float64
 1   Region_Code              50882 non-null  int64  
 2   Accomodation_Type        50882 non-null  float64
 3   Reco_Insurance_Type      50882 non-null  float64
 4   Upper_Age                50882 non-null  int64  
 5   Lower_Age                50882 non-null  int64  
 6   Is_Spouse                50882 non-null  float64
 7   Health Indicator         50882 non-null  float64
 8   Holding_Policy_Duration  50882 non-null  float64
 9   Holding_Policy_Type      50882 non-null  float64
 10  Reco_Policy_Cat          50882 non-null  int64  
 11  Reco_Policy_Premium      50882 non-null  float64
 12  prembin                  50882 non-null  float64
dtypes: float64(9), int64(4)
memory usage: 5.0 MB


Under Sampling

In [19]:
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(random_state=42, replacement=True)
# x_rus, y_rus = rus.fit_resample(x_train_encoded, y)

# print('original dataset shape:', x_rus.shape)
# print('Resample dataset shape', x_train_encoded.shape)

In [20]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()
x_rus, y_rus = smote.fit_resample(x_train_encoded, y)

print('original dataset shape:', x_rus.shape)
print('Resample dataset shape', x_train_encoded.shape)

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


original dataset shape: (77346, 13)
Resample dataset shape (50882, 13)


In [21]:
# # import library
# from imblearn.over_sampling import RandomOverSampler
# ros = RandomOverSampler(random_state=42)
# x_rus, y_rus = ros.fit_resample(x_train_encoded, y)

In [22]:
x,y = x_rus,y_rus

In [23]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score
from sklearn.metrics import roc_auc_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb 
import xgboost as xgb
from collections import Counter
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

In [24]:
x_train, x_test, y_train, y_test = train_test_split(x, y, shuffle = True, test_size = 0.2)
print(x_train.shape, x_test.shape)
print(y_train.shape, y_test.shape)
print(test_encoded.shape)

(61876, 13) (15470, 13)
(61876,) (15470,)
(21805, 13)


In [25]:
test = test_encoded

In [27]:
!pip install catboost

     |████████████████████████████████| 65.7MB 70kB/s 


In [28]:
print('Random Forest Classifier')
clf=RandomForestClassifier()
clf.fit(x_train,y_train)
y_predrf=clf.predict(x_test)
print(roc_auc_score(y_test,y_predrf),'\n')

print('Gradient Boosting Classifier')
model = GradientBoostingClassifier()
model.fit(x_train, y_train)
ypredgbc = model.predict(x_test)
print(roc_auc_score(y_test,ypredgbc),'\n')

from xgboost import XGBClassifier
print('XGB Classifier')
model = XGBClassifier()
model.fit(x_train, y_train)
ypredxgb = model.predict(x_test)
print(roc_auc_score(y_test,ypredxgb),'\n')

from lightgbm import LGBMClassifier
print('LGBM Classifier')
model = LGBMClassifier()
model.fit(x_train, y_train)
ypredlgbm = model.predict(x_test)
print(roc_auc_score(y_test,ypredlgbm),'\n')

from catboost import CatBoostClassifier
print('CATBoost Classifier')
model = CatBoostClassifier(verbose=0, n_estimators=100)
model.fit(x_train,y_train)
ypredcat = model.predict(x_test)
print(roc_auc_score(y_test,ypredcat),'\n')

Random Forest Classifier
0.8361521934397284 

Gradient Boosting Classifier
0.8369687251449425 

XGB Classifier
0.8381998452468534 

LGBM Classifier
0.8439661475360691 

CATBoost Classifier
0.8423627787709145 



In [ ]:
#LGBM Final Submission 
print('LGBM Classifier')
model = LGBMClassifier()
model.fit(x, y)
ypredlgbm = model.predict_proba(test)

In [ ]:
# df = pd.DataFrame(ypredlgbm)
# from google.colab import files
# df.to_csv('lgbmsmotetarget.csv') 
# files.download('lgbmsmotetarget.csv')